## MediaPipe Simple Usage

On Ubuntu, `ls /dev/video*` displays available camera devices.

In [ ]:
%ls /dev/video*

In [ ]:
import cv2
import mediapipe as mp
import time

mp_facedetector = mp.solutions.face_detection
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(4)
# 2 4 6

#cap.release()

### Hand Landmarks

In [ ]:
hands = mp.solutions.hands
hands_mesh = hands.Hands(static_image_mode=True, min_detection_confidence=0.7)
draw = mp.solutions.drawing_utils

while True:
    success, bgr = cap.read()
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

    results = hands_mesh.process(rgb)

    if results.multi_hand_landmarks:
        for i in results.multi_hand_landmarks:
            draw.draw_landmarks(bgr, i, hands.HAND_CONNECTIONS, 
                landmark_drawing_spec=draw.DrawingSpec(color = (255, 0,0),circle_radius=4, thickness=3),
                connection_drawing_spec=draw.DrawingSpec(thickness=3, color=(0,0,255)))

    cv2.imshow("Hand Landmarks", bgr)

    key = cv2.waitKey(5)
    if key & 0xFF == 27 or key == ord('q'):
        cv2.destroyAllWindows()
        break


### Face Detection

In [ ]:
face_detection = mp_facedetector.FaceDetection(min_detection_confidence=0.7)

while cap.isOpened():

    success, bgr = cap.read()
    start = time.time()

    # Convert the BGR image to RGB
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

    # Process the image and find faces
    results = face_detection.process(rgb)


    # If detected, put
    if results.detections:
        for id, detection in enumerate(results.detections):

            mp_draw.draw_detection(bgr, detection)
            #print(id, detection)

            bBox = detection.location_data.relative_bounding_box
            h, w, c = bgr.shape
            boundBox = int(bBox.xmin * w), int(bBox.ymin * h), int(bBox.width * w), int(bBox.height * h)

            cv2.putText(bgr, f'{int(detection.score[0]*100)}%', (boundBox[0], boundBox[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 2)


    end = time.time()
    totalTime = end - start
    fps = 1 / totalTime
    cv2.putText(bgr, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

    cv2.imshow('Face Detection', bgr)

    key = cv2.waitKey(5)
    if key & 0xFF == 27 or key == ord('q'):
        cv2.destroyAllWindows()
        break
